In [1]:
%pip install plotly --quiet
%pip install pandas --quiet
%pip install ta --quiet
import ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
import json
import datetime
import pandas as pd
import numpy as np
import time

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Kraken data, some of this setup is specific to Kraken.
pair = "XBTUSDT"
time_interval_in_minutes = 60
url = f"https://api.kraken.com/0/public/OHLC?pair={pair}&interval={time_interval_in_minutes}"
response = requests.get(url)
data = json.loads(response.text)

ohlc_data = data['result'][pair]

# Create a DataFrame from the OHLC data
df = pd.DataFrame(ohlc_data, columns=['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count'])
df['time'] = pd.to_datetime(df['time'], unit='s')
df[['open', 'high', 'low', 'close', 'vwap', 'volume', 'count']] = df[['open', 'high', 'low', 'close', 'vwap', 'volume', 'count']].apply(pd.to_numeric)
df.drop(['vwap', 'volume', 'count'], axis=1, inplace=True)

# Calculate some indicators.
df['atr'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=14)
df['rsi'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
df['sma14'] = ta.trend.sma_indicator(df['close'], window=14)
df[['atr', 'rsi', 'sma14']] = df[['atr', 'rsi', 'sma14']].round(2)

# Drop first rows where the indicators couldn't be calculated due to lack of data. Don't really need to drop I think, but I did.
df.drop(df.index[:14], inplace=True)
df.reset_index(drop=True, inplace=True)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

In [3]:
df

,time,open,high,low,close,atr,rsi,sma14
0,2023-05-05 05:00:00,29230.8,29244.7,29169.6,29209.9,148.76,66.96,28991.04
1,2023-05-05 06:00:00,29207.6,29234.4,29152.1,29155.9,144.01,61.98,29007.78
2,2023-05-05 07:00:00,29155.9,29238.1,29053.6,29056.4,146.91,54.02,29025.21
3,2023-05-05 08:00:00,29066.8,29118.6,29051.5,29061.4,141.21,54.33,29043.67
4,2023-05-05 09:00:00,29061.2,29144.2,29040.7,29077.9,138.51,55.43,29057.58
...,...,...,...,...,...,...,...,...
701,2023-06-03 10:00:00,27147.6,27171.3,27132.8,27144.3,68.78,51.70,27175.23
702,2023-06-03 11:00:00,27145.7,27171.4,27129.2,27136.7,66.88,50.88,27169.28
703,2023-06-03 12:00:00,27136.8,27171.4,27136.8,27170.5,64.58,54.33,27162.94
704,2023-06-03 13:00:00,27170.5,27185.3,27148.7,27171.4,62.58,54.42,27157.32


In [4]:
########### find_horizontal_resistance_lines ###########

#--> Identifies an "origin" green candle for resistance line starting point. 
#--> Defines the resistance 'line_price' as a weighted mean of the origin candle's high and close prices. 
# (This is superior to HLC/3 because we are only interested in the high wicks for resistance lines). 
#--> The 'waiting_period' and the 'waiting_candles' is a way to separate 'clumped' candle touches to have min # candles between origin point and 2nd touch. 
#--> Checks the following candles for touch points, using % of atr as wiggle room to define an acceptable range for touch points.
#--> Makes sure there are no overlapping lines during line creation.

########### find_horizontal_resistance_lines ###########

In [4]:
def find_horizontal_resistance_lines(df):
    resistance_lines = []
    i = 0
    waiting_period = 10

    # Start a loop over the DataFrame, leaving space for checking a waiting period.
    while i < len(df) - waiting_period: 
        origin_candle = df.iloc[i]

        # Only consider green candles as potential origin points. Could also consider red ones and pick max of 'close', 'open', but isn't as clean.
        if origin_candle['close'] > origin_candle['open']:
            # Define line price as weighted mean of close and high. Set 'second_touch' flag to None.
            line_price = 0.6 * origin_candle['close'] + 0.4 * origin_candle['high'] 
            second_touch = None 

            # Define an acceptable range for the waiting candles. Uses 'atr' of current iterated candle. 
            waiting_candles = df.iloc[i+1:i+waiting_period+1]
            waiting_candles_upper_range = line_price + (0.5 * waiting_candles['atr'])
            waiting_candles_line_price = 0.6 * waiting_candles[['close', 'open']].max(axis=1) + 0.4 * waiting_candles['high']
            
            # Skip to the next 'origin' candle if any waiting candle's close prices exceed the upper limit.
            if any(waiting_candles_line_price > waiting_candles_upper_range):
                i += 1
                continue

            # If all waiting candles are within the range, start looking for the second touch point. Uses 'atr' of current iterated candle.   
            for j in range(i+waiting_period, len(df)):  
                candle = df.iloc[j]

                # Define the upper and lower acceptable touch ranges for the next candle. 
                # 'bottom_limit' is useful for if the price goes way below resistance line, stop the line.
                upper_range = line_price + (0.5 * candle['atr'])
                lower_range = line_price - (0.5 * candle['atr'])
                bottom_limit = line_price - (5 * candle['atr'])

                # 'candle' are the candles when looking for the 2nd touch point.
                # 'bottom' refers to the weighted line price we are going to use when looking for if the 'bottom_limit' gets hit.
                candle_line_price = 0.6 * candle[['close', 'open']].max() + 0.4 * candle['high']
                candle_line_price_bottom = 0.6 * candle[['close', 'open']].min() + 0.4 * candle['low']

                # Check if the candle is a touch point.
                if lower_range <= candle_line_price <= upper_range and second_touch is None:
                    second_touch = candle['time']
                    touch_2_price = candle_line_price
                
                # Check if the candle is outside of the acceptable range or bottom_limit
                if candle_line_price > upper_range or candle_line_price_bottom < bottom_limit:
                    # If there was a second touch, save the line to the list of resistance lines.
                    if second_touch is not None:
                        resistance_lines.append({'initial_point': origin_candle['time'], 'end_of_line': candle['time'], 'touch_2': second_touch, 'touch_2_price': touch_2_price, 'line_price': line_price})
                    second_touch = None  
                    i = j + 1  # Start analyzing from the next candle
                    break
            else:
                i += 1  # If the loop finishes without breaking, check the next candle.
        else:
            i += 1  # If the candle is not green, check the next one.

    return pd.DataFrame(resistance_lines)

resistance_df = find_horizontal_resistance_lines(df)

In [5]:
resistance_df['initial_point'] = pd.to_datetime(resistance_df['initial_point'])
resistance_df['end_of_line'] = pd.to_datetime(resistance_df['end_of_line'])

# Calculates length of horizontal resistance line based on time interval specified while loading Kraken data. Returns # of candles as a value.
resistance_df['line_length'] = (resistance_df['end_of_line'] - resistance_df['initial_point']).dt.total_seconds() / (time_interval_in_minutes * 60)
resistance_df['line_length'] = resistance_df['line_length'].astype(int)

In [6]:
resistance_df

,initial_point,end_of_line,touch_2,touch_2_price,line_price,line_length
0,2023-05-07 00:00:00,2023-05-07 14:00:00,2023-05-07 12:00:00,28925.92,28942.60,14
1,2023-05-13 00:00:00,2023-05-13 18:00:00,2023-05-13 10:00:00,26867.70,26825.50,18
2,2023-05-13 19:00:00,2023-05-14 14:00:00,2023-05-14 06:00:00,26870.32,26891.72,19
3,2023-05-15 06:00:00,2023-05-15 17:00:00,2023-05-15 16:00:00,27514.46,27495.60,11
4,2023-05-17 19:00:00,2023-05-18 17:00:00,2023-05-18 07:00:00,27414.98,27387.30,22
5,2023-05-18 21:00:00,2023-05-20 17:00:00,2023-05-19 07:00:00,26921.16,26970.08,44
6,2023-05-23 03:00:00,2023-05-24 03:00:00,2023-05-23 14:00:00,27335.70,27384.10,24
7,2023-05-24 04:00:00,2023-05-27 23:00:00,2023-05-26 14:00:00,26833.62,26797.98,91
8,2023-05-31 06:00:00,2023-05-31 23:00:00,2023-05-31 20:00:00,27112.10,27171.84,17
9,2023-06-01 01:00:00,2023-06-02 08:00:00,2023-06-01 16:00:00,27133.26,27108.82,31


In [7]:
########### find_touch_points ###########

#--> After the initial two points of a resistance line are established, this function finds further touch points.
#--> Utilizes a 'waiting_period' to ensure some distance between subsequent touch points, preventing clumping.
#--> Defines the acceptable range for touch points using a % of atr as a threshold. Checks the df for touch points within this range.
#--> If a touch point is found, the function records it's price and timestamp, and starts the search for the next touch point after the waiting period.
#--> If the price moves outside of the acceptable range before another touch point is found, or if the 'bottom_limit' is crossed, the line ends. 
#--> Iterates over the resistance_df until all lines have been evaluated for further touch points.

########### find_touch_points ###########

In [7]:
def find_touch_points(df, resistance_df):

    # Set waiting period between touches, initialize counter for max touch points, initialize list of column names for touch points and prices.
    waiting_period = 10
    max_touch_points = 0
    touch_point_cols = []
    touch_price_cols = []
    
    # Iterate over the resistance lines DataFrame.
    for idx, row in resistance_df.iterrows():
        # Define the start index (just after the second touch point), end index and the resistance line price.
        second_touch_point = row['touch_2']
        price = row['line_price']
        start_idx = df.loc[df['time'] == second_touch_point].index[0] + waiting_period
        end_idx = df.loc[df['time'] == row['end_of_line']].index[0]

        # Initialize dict for touch points and prices.
        touch_points = {}
        touch_prices = {}

        # Loop over the df from the start index to the end index.
        while start_idx < end_idx:
            slice_df = df.iloc[start_idx:end_idx+1]  

            # Initialize a flag for found touch points.
            found = False

            # Loop over each candle within the sliced DataFrame.
            for j, candle in slice_df.iterrows():
                # Define the upper and lower acceptable ranges for touch points.
                # 'bottom_limit' is useful for if the price goes way below resistance line, stop the line.
                upper_range = price + (0.5 * candle['atr'])
                lower_range = price - (0.5 * candle['atr'])
                bottom_limit = price - (5 * candle['atr'])

                candle_line_price = 0.6 * candle[['close', 'open']].max() + 0.4 * candle['high']
                candle_line_price_bottom = 0.6 * candle[['close', 'open']].min() + 0.4 * candle['low']

                # Check if the candle is within the acceptable range for a touch point.
                if lower_range <= candle_line_price <= upper_range:
                    touch_points[len(touch_points)+3] = candle['time']
                    touch_prices[len(touch_prices)+3] = candle_line_price
                    # Start the next search after the waiting period.
                    start_idx = j + waiting_period
                    found = True
                    break
                # Check if the candle is outside of the acceptable range or bottom_limit.
                if candle_line_price > upper_range or candle_line_price_bottom < bottom_limit:
                    break
            # If no touch point was found, break the loop.                
            if not found:
                break

        # Update the maximum number of touch points.
        max_touch_points = max(max_touch_points, len(touch_points))

        # Loops over found touch points and adds them to resistance_df.
        for i, touch_point in touch_points.items():
            # Define column names for touch points and their prices.
            col_name = f'touch_point_{i}'
            price_col_name = f'touch_point_{i}_price'

            # Add new columns (timestamp and price) to the DataFrame if they don't exist.
            if col_name not in touch_point_cols:
                touch_point_cols.append(col_name)
                resistance_df[col_name] = None
            if price_col_name not in touch_price_cols:
                touch_price_cols.append(price_col_name)
                resistance_df[price_col_name] = None

            # Add the touch points (timestamps) and their prices to the df.
            resistance_df.loc[idx, col_name] = touch_point
            resistance_df.loc[idx, price_col_name] = touch_prices[i]
        # Record the total number of touch points for the line in the df.
        resistance_df.loc[idx, 'num_touch_points'] = len(touch_points)

    return resistance_df

resistance_df = find_touch_points(df, resistance_df)

In [8]:
# Creates 3rd touch boolean for calculating returns later. 
resistance_df['3rd_touch_bool'] = resistance_df['touch_point_3'].notna()
# Every line has minimum of 2 touchpoints already at this point.
resistance_df['num_touch_points'] = resistance_df['num_touch_points'] + 2 
resistance_df['num_touch_points'] = resistance_df['num_touch_points'].astype(int)

In [10]:
########### calculate_returns_after_third_touch ###########

#--> Calculates the returns after 3rd touch point of a resistance line. Only processes resistance lines that have 3rd touch point.
#--> Uses the closing price at the third touch point as the entry point for the trade.
#--> The data is sliced from the 3rd touch point to the end of a period desired period (3*line_length).
#--> Returns calculated as 'entry_price' - closing candles prices.
#--> The return data is stored in a dictionary indexed by the resistance line ID. 
#--> Good for determining a trading strategy or backtesting/optimizing for SL/TP/etc. (Use bulk data rather than this sample Kraken data)

########### calculate_returns_after_third_touch ###########

In [9]:
def calculate_returns_after_third_touch(df, resistance_df):
    returns_dict = {}

    for idx, row in resistance_df.iterrows():
        if not row['3rd_touch_bool']:
            continue

        line_length = row['line_length']
        third_point = row['touch_point_3']
        
        # Define the entry price as the closing price at the third touch point. (May be a bit below or above the 'line_price' in earlier functions.)
        entry_price = df.loc[df['time'] == third_point, 'close'].values[0]

        # Define the start and end indices for gathering returns data (3x length of line).
        start_idx = df.loc[df['time'] == third_point].index[0]
        end_idx = min(start_idx + 3 * line_length, len(df) - 1)
        # Slice df from the third touch point to the end of the period.
        slice_df = df.iloc[start_idx:end_idx+1]
        # Calculate returns based on the entry price and store them in the dictionary.
        returns = slice_df['close'].apply(lambda x: x - entry_price)
        returns_dict[idx] = returns.tolist()

    return returns_dict

returns_dict = calculate_returns_after_third_touch(df, resistance_df)

In [12]:
########### calculate_trading_results ###########

#--> Calculates the trading results based on returns calculated.
#--> Uses the closing price as the 3rd touch point as the entry point for the trade.
#--> Length of trade is up to 3x length of resistance line.
#--> Defines the stop loss level as 1.3*atr below the entry, and the take profit level as 3*atr above the entry.
#--> Iterates through the sliced data to determine whether the stop loss or take profit was hit first, or neither was hit during the period.
#--> Adds columns to the resistance line DataFrame to store the entry price, stop loss, take profit, the result, timestamp of target hit

########### calculate_trading_results ###########

In [10]:
def calculate_trading_results(df, resistance_df):
    for idx, row in resistance_df.iterrows():
        if not row['3rd_touch_bool']:
            continue

        third_point = row['touch_point_3']
        line_length = row['line_length']
        # Define the entry price as the closing price at the third touch point. 
        entry_price = df.loc[df['time'] == third_point, 'close'].values[0]
        # Define the start time (3rd touch point) and end indices for gathering returns data (3x the line length).
        start_idx = df.loc[df['time'] == third_point].index[0]
        end_idx = min(start_idx + 3 * line_length, len(df) - 1)
        # Slice the DataFrame from the third touch point to the end of the period.
        slice_df = df.iloc[start_idx:end_idx+1]

        result = 'None'
        target_hit_time = 'None'

        # Iterate through the sliced DataFrame to check whether the stop loss or take profit was hit.
        for _, candle in slice_df.iterrows():
            atr = candle['atr']
            sl_level = entry_price - 1.3 * atr
            tp_level = entry_price + 3 * atr

            if candle['high'] >= tp_level:
                result = 'Take Profit'
                target_hit_time = candle['time']
                break
            elif candle['low'] <= sl_level:
                result = 'Stop Loss'
                target_hit_time = candle['time']
                break

        if result == 'None':
            result = 'No Target Hit During Period'

        resistance_df.at[idx, 'entry_price'] = entry_price
        resistance_df.at[idx, 'sl'] = sl_level
        resistance_df.at[idx, 'tp'] = tp_level
        resistance_df.at[idx, 'result'] = result
        resistance_df.at[idx, 'target_hit_time'] = target_hit_time

calculate_trading_results(df, resistance_df)

In [11]:
resistance_df['touch_point_3'] = pd.to_datetime(resistance_df['touch_point_3'])
resistance_df['target_hit_time'] = pd.to_datetime(resistance_df['target_hit_time'])
# Calculates time in trade using the interval from Kraken data. Returns # of candles.
resistance_df['time_in_trade'] = (resistance_df['target_hit_time'] - resistance_df['touch_point_3']).dt.total_seconds() / (time_interval_in_minutes * 60)
resistance_df['time_in_trade'] = resistance_df['time_in_trade'].astype('Int64')
# Calculates net_pl if sl or tp was hit.
def calculate_net_pl(row):
    if row['result'] == 'Take Profit':
        return row['tp'] - row['entry_price']
    elif row['result'] == 'Stop Loss':
        return row['sl'] - row['entry_price']
    else:
        return np.nan
resistance_df['net_pl'] = resistance_df.apply(calculate_net_pl, axis=1)

In [12]:
# Sorting df the way I want (automatically takes care of N touch points) (lmao)

# Define the manually ordered columns before and after touch points
manual_cols_before = ['initial_point', 'end_of_line', 'line_price', 'line_length', 'touch_2', 'touch_2_price']
manual_cols_after = ['num_touch_points', '3rd_touch_bool', 'entry_price', 'sl', 'tp', 'result', 'target_hit_time', 'time_in_trade', 'net_pl']
# Extract all columns starting with 'touch_point_' and sort them
touch_cols = sorted([col for col in resistance_df.columns if col.startswith('touch_point_') and col not in manual_cols_before and col not in manual_cols_after])
# Combine all lists while maintaining the desired order
final_cols = manual_cols_before + touch_cols + manual_cols_after
# Reorganize the DataFrame using the new column order
resistance_df = resistance_df[final_cols]
resistance_df

,initial_point,end_of_line,line_price,line_length,touch_2,touch_2_price,touch_point_3,touch_point_3_price,touch_point_4,touch_point_4_price,num_touch_points,3rd_touch_bool,entry_price,sl,tp,result,target_hit_time,time_in_trade,net_pl
0,2023-05-07 00:00:00,2023-05-07 14:00:00,28942.60,14,2023-05-07 12:00:00,28925.92,NaT,None,None,None,2,False,NaN,NaN,NaN,NaN,NaT,<NA>,NaN
1,2023-05-13 00:00:00,2023-05-13 18:00:00,26825.50,18,2023-05-13 10:00:00,26867.70,NaT,None,None,None,2,False,NaN,NaN,NaN,NaN,NaT,<NA>,NaN
2,2023-05-13 19:00:00,2023-05-14 14:00:00,26891.72,19,2023-05-14 06:00:00,26870.32,NaT,None,None,None,2,False,NaN,NaN,NaN,NaN,NaT,<NA>,NaN
3,2023-05-15 06:00:00,2023-05-15 17:00:00,27495.60,11,2023-05-15 16:00:00,27514.46,NaT,None,None,None,2,False,NaN,NaN,NaN,NaN,NaT,<NA>,NaN
4,2023-05-17 19:00:00,2023-05-18 17:00:00,27387.30,22,2023-05-18 07:00:00,27414.98,NaT,None,None,None,2,False,NaN,NaN,NaN,NaN,NaT,<NA>,NaN
5,2023-05-18 21:00:00,2023-05-20 17:00:00,26970.08,44,2023-05-19 07:00:00,26921.16,2023-05-19 17:00:00,26933.16,2023-05-20 15:00:00,26952.18,4,True,26922.0,26835.901,27120.69,Take Profit,2023-05-20 17:00:00,24,198.690
6,2023-05-23 03:00:00,2023-05-24 03:00:00,27384.10,24,2023-05-23 14:00:00,27335.70,NaT,None,None,None,2,False,NaN,NaN,NaN,NaN,NaT,<NA>,NaN
7,2023-05-24 04:00:00,2023-05-27 23:00:00,26797.98,91,2023-05-26 14:00:00,26833.62,2023-05-27 03:00:00,26763.7,2023-05-27 17:00:00,26761.6,4,True,26753.9,26653.306,26986.04,Stop Loss,2023-05-27 13:00:00,10,-100.594
8,2023-05-31 06:00:00,2023-05-31 23:00:00,27171.84,17,2023-05-31 20:00:00,27112.10,NaT,None,None,None,2,False,NaN,NaN,NaN,NaN,NaT,<NA>,NaN
9,2023-06-01 01:00:00,2023-06-02 08:00:00,27108.82,31,2023-06-01 16:00:00,27133.26,2023-06-02 04:00:00,27125.4,None,None,3,True,27085.2,26914.289,27479.61,Stop Loss,2023-06-02 13:00:00,9,-170.911


In [13]:
trades_df = resistance_df[resistance_df['entry_price'].notnull()]
trades_df

,initial_point,end_of_line,line_price,line_length,touch_2,touch_2_price,touch_point_3,touch_point_3_price,touch_point_4,touch_point_4_price,num_touch_points,3rd_touch_bool,entry_price,sl,tp,result,target_hit_time,time_in_trade,net_pl
5,2023-05-18 21:00:00,2023-05-20 17:00:00,26970.08,44,2023-05-19 07:00:00,26921.16,2023-05-19 17:00:00,26933.16,2023-05-20 15:00:00,26952.18,4,True,26922.0,26835.901,27120.69,Take Profit,2023-05-20 17:00:00,24,198.690
7,2023-05-24 04:00:00,2023-05-27 23:00:00,26797.98,91,2023-05-26 14:00:00,26833.62,2023-05-27 03:00:00,26763.7,2023-05-27 17:00:00,26761.6,4,True,26753.9,26653.306,26986.04,Stop Loss,2023-05-27 13:00:00,10,-100.594
9,2023-06-01 01:00:00,2023-06-02 08:00:00,27108.82,31,2023-06-01 16:00:00,27133.26,2023-06-02 04:00:00,27125.4,None,None,3,True,27085.2,26914.289,27479.61,Stop Loss,2023-06-02 13:00:00,9,-170.911


In [14]:
# Candlestick plot
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Plots candlestick graph with atr, rsi, sma, horizontal resistance line, all touch points and target hit point.
candlestick_trace = go.Candlestick(x=df['time'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], name='Candlesticks')
atr_trace = go.Scatter(x=df['time'], y=df['atr'], mode='lines', name='ATR', line=dict(color='purple'), opacity=0.2)
first_touch_trace = go.Scatter(x=resistance_df['initial_point'], y=resistance_df['line_price'], mode='markers', name='First Touch Point', marker=dict(color='white', size=6))
second_touch_trace = go.Scatter(x=resistance_df['touch_2'], y=resistance_df['touch_2_price'], mode='markers', name='Second Touch Point', marker=dict(color='white', size=6))
target_trace = go.Scatter(x=resistance_df['target_hit_time'], y=resistance_df['net_pl'] + resistance_df['entry_price'], mode='markers', name='Target Hit', marker=dict(color='#FFD700', size=7))
sma_trace = go.Scatter(x=df['time'], y=df['sma14'], mode='lines', name='SMA14', line=dict(color='#87CEEB'), opacity=0.4)
rsi_trace = go.Scatter(x=df['time'], y=df['rsi'], mode='lines', name='RSI', line=dict(color='#32CD32'), opacity=0.2)

fig.add_trace(candlestick_trace, secondary_y=False)
fig.add_trace(first_touch_trace, secondary_y=False)
fig.add_trace(second_touch_trace, secondary_y=False)

# adding multiple touch point traces to the figure starting with 'touch_point_3'.
for i in range(3, len(resistance_df.columns)):
    touch_point_column = 'touch_point_' + str(i)
    touch_price_column = 'touch_point_' + str(i) + '_price'
    # If a column name exists in the df in the format 'touch_point_i', a trace is created for the touch points in that column and added to graph.
    if touch_point_column in resistance_df.columns:
        touch_point_trace = go.Scatter(x=resistance_df[touch_point_column], y=resistance_df[touch_price_column], mode='markers', name=f'Touch Point {i}', marker=dict(color='#B200ED', size=7))
        fig.add_trace(touch_point_trace, secondary_y=False)

fig.add_trace(target_trace, secondary_y=False)
fig.add_trace(sma_trace, secondary_y=False)
fig.add_trace(atr_trace, secondary_y=True)
fig.add_trace(rsi_trace, secondary_y=True)

fig.update_layout(title='BTC/USDT', xaxis_title='Time', template="plotly_dark")
fig.update_yaxes(title_text='Price', secondary_y=False, tickformat="$,.2f", autorange=True, fixedrange=False)
fig.update_yaxes(title_text='ATR', secondary_y=True)

# Adds the horizontal resistance lines to graph. Iterates through each row of the resistance_df.
for _, row in resistance_df.iterrows():
    start_time = row['initial_point']
    end_time = row['end_of_line']
    price = row['line_price']
    
    fig.add_shape(type="line", x0=start_time, y0=price, x1=end_time, y1=price, line=dict(color='#39FF14', width=2), xref='x', yref='y')

fig.show()

# Returns plot
fig = go.Figure()

# Creates a trace for each item in the returns_dict and adds them to the second figure.
# Each item in returns_dict represents the return series from a particular resistance line, starting from the third touch point.
for idx, (_, returns) in enumerate(returns_dict.items()):
    scatter_trace = go.Scatter(y=returns, mode='lines', name=f'res_line_{idx + 1}', line=dict(color='#FFD700'))
    fig.add_trace(scatter_trace)

fig.add_hline(y=0, line=dict(color='#39FF14', width=2, dash='dash'))
fig.update_layout(title="Returns from 3rd Touch Point, Long Entry is Close of 3rd Touch Point", xaxis_title="# Candles after 3rd Touch", yaxis_title="Returns", template="plotly_dark")
fig.show()